In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os


# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
dataset_path = "/home/kevin/datasets/netflix/"

In [3]:
read = dataset_path + "/README"
! cat $read

In [4]:
qualify = dataset_path + "/qualifying.txt"
! head $qualify

1:
1046323,2005-12-19
1080030,2005-12-23
1830096,2005-03-14
368059,2005-05-26
802003,2005-11-07
513509,2005-07-04
1086137,2005-09-21
428698,2005-12-20
515850,2005-11-27


In [4]:
"""
# index by both movies and users
current_movie = 0
data_by_movie = {}

for i in range(1, 5):
    print(i)
    with open(dataset_path + '/combined_data_' + str(i) + '.txt') as f:
        for line in f:
            line = line.split(',')
            if len(line) == 1:
                current_movie = line[0][:-2]
                data_by_movie[current_movie] = {}
            else:
                current_user = line[0]
                data_by_movie[current_movie][current_user] = line[1:]
        
with open('data_by_movie_all.npy', 'wb') as f:
    np.save(f, data_by_movie)
"""

"\n# index by both movies and users\ncurrent_movie = 0\ndata_by_movie = {}\n\nfor i in range(1, 5):\n    print(i)\n    with open(dataset_path + '/combined_data_' + str(i) + '.txt') as f:\n        for line in f:\n            line = line.split(',')\n            if len(line) == 1:\n                current_movie = line[0][:-2]\n                data_by_movie[current_movie] = {}\n            else:\n                current_user = line[0]\n                data_by_movie[current_movie][current_user] = line[1:]\n        \nwith open('data_by_movie_all.npy', 'wb') as f:\n    np.save(f, data_by_movie)\n"

In [5]:
data = np.load('data_by_movie.npy', allow_pickle=True)

In [6]:
true_data = data.item()

In [7]:
len(true_data.keys())

4499

In [10]:
"""
with open('data_by_movie.npy', 'wb') as f:
    np.save(f, data_by_movie)
    
with open('data_by_user.npy', 'wb') as f:
    np.save(f, data_by_user)

len(data_by_movie.keys())

len(data_by_user.keys())

data_by_movie['1']

ratings = []
for movie in data_by_movie.keys():
    for user in data_by_movie[movie].keys():
        ratings.append(data_by_movie[movie][user][0])

average_rating = np.average(np.array([int(x) for x in ratings]))
"""

"\nwith open('data_by_movie.npy', 'wb') as f:\n    np.save(f, data_by_movie)\n    \nwith open('data_by_user.npy', 'wb') as f:\n    np.save(f, data_by_user)\n\nlen(data_by_movie.keys())\n\nlen(data_by_user.keys())\n\ndata_by_movie['1']\n\nratings = []\nfor movie in data_by_movie.keys():\n    for user in data_by_movie[movie].keys():\n        ratings.append(data_by_movie[movie][user][0])\n\naverage_rating = np.average(np.array([int(x) for x in ratings]))\n"

In [35]:
import numpy.linalg as LA
def min_function(r, q, p, lamb, cap=1000000):
    count = 0
    total = 0
    for movie in r.keys():
        for user in r[movie].keys():
            count += 1
            rating = float(r[movie][user][0])
            if movie not in q:
                q[movie] = np.random.rand(latent_dim,1)
            if user not in p:
                p[user] = np.random.rand(latent_dim,1)
            prediction = np.dot(q[movie].T, p[user]) 
            norm_q_i = LA.norm(q[movie]) 
            norm_p_u = LA.norm(p[user])
            total += (rating-prediction)**2 + lamb*(norm_q_i + norm_p_u)
            if count % 10000 == 0:
                print("Count", count, "Total", total)
        if count >= cap:
            break


In [38]:


latent_dim = 5

q = {}
p = {}
min_function(data_by_movie, q, p, 0.1)




Count 10000 Total [[59593.5490582]]
Count 20000 Total [[108700.60107372]]
Count 30000 Total [[158908.00455203]]
Count 40000 Total [[206774.06589883]]
Count 50000 Total [[260879.72382624]]
Count 60000 Total [[330506.82158083]]
Count 70000 Total [[404477.46730154]]
Count 80000 Total [[478327.86354923]]
Count 90000 Total [[552430.43961205]]
Count 100000 Total [[644005.1864228]]
Count 110000 Total [[742516.99364892]]
Count 120000 Total [[840478.9446101]]
Count 130000 Total [[939247.36110412]]
Count 140000 Total [[1038014.02077253]]
Count 150000 Total [[1136154.839783]]
Count 160000 Total [[1234119.93136704]]
Count 170000 Total [[1332063.32693328]]
Count 180000 Total [[1429880.22411054]]
Count 190000 Total [[1528323.4838531]]
Count 200000 Total [[1625849.60663246]]
Count 210000 Total [[1723374.75193003]]
Count 220000 Total [[1829083.30150738]]
Count 230000 Total [[1890949.07875622]]
Count 240000 Total [[1946497.79013118]]
Count 250000 Total [[2004552.61094575]]
Count 260000 Total [[2084736.

In [39]:
def sgd_function(r, q, p, lamb, lr):
    count = 0
    total = 0
    for movie in r.keys():
        for user in r[movie].keys():
            count += 1
            rating = float(r[movie][user][0])
            prediction = np.dot(q[movie].T, p[user]) 
            e_ui = rating-prediction
            updated_q_i = q[movie] + lr*(e_ui*p[user] - lamb*q[movie])
            updated_p_u = p[user] + lr*(e_ui*q[movie] - lamb*p[user])
            q[movie] = updated_q_i
            p[user] = updated_p_u
            # should SGD be all after? Let's do present for now
        if count >= 1000000:
            break
            
sgd_function(data_by_movie, q, p, 0.1, 0.1)

In [36]:
import pdb
def als_function1(r, d, x, beta, lamb, latent_dim=5, default = 1, cap=100000):
    count = 0
    total = 0
    for a in r.keys():
        X = []
        y = []
        print("default", default, "beta", a)
        for b in r[a].keys():
            count += 1
            rating =float(d[a][b][0]) if default == 1 else float(d[b][a][0])
            if b not in x:
                x[b] = np.random.rand(latent_dim,1)
            X.append(x[b])
            y.append(rating)
        X = np.array(X)
        y = np.array(y)
        #pdb.set_trace()
        y = np.expand_dims(y, axis=1)
        X = np.reshape(X, (len(X), latent_dim))
        beta[b] = np.linalg.inv(X.T @ X + lamb) @ (X.T @ y)
        if count >= cap:
            break

q = {}
p = {}
min_function(true_data, q, p, 0.1, cap=100000)
als_function1(true_data, true_data, q, p, 0.1, cap=100000)
#als_function1(user_dictionary, true_data, p, q, 0.1, default=0)
min_function(true_data, q, p, 0.1, cap=100000)


Count 10000 Total [[65887.87629862]]
Count 20000 Total [[112220.62452351]]
Count 30000 Total [[155335.52746147]]
Count 40000 Total [[235583.65149902]]
Count 50000 Total [[294168.72247561]]
Count 60000 Total [[364331.8293926]]
Count 70000 Total [[445304.89474313]]
Count 80000 Total [[526056.67361766]]
Count 90000 Total [[607030.92210497]]
Count 100000 Total [[677485.82146999]]
Count 110000 Total [[744597.68061007]]
Count 120000 Total [[811739.23781468]]
Count 130000 Total [[879248.6475429]]
Count 140000 Total [[946936.28197575]]
Count 150000 Total [[1013596.40079789]]
Count 160000 Total [[1080624.72880615]]
Count 170000 Total [[1147111.16023591]]
Count 180000 Total [[1213751.57993953]]
Count 190000 Total [[1280960.16399142]]
Count 200000 Total [[1347518.60051301]]
Count 210000 Total [[1413936.6161543]]
default 1 beta 1
default 1 beta 2
default 1 beta 3
default 1 beta 4
default 1 beta 5
default 1 beta 6
default 1 beta 7
default 1 beta 8
default 1 beta 9
default 1 beta 10
default 1 beta 1

In [21]:
len(true_data['5'].values())

1140

In [40]:
min_function(data_by_movie, q, p, 0.1)

Count 10000 Total [[21681.69218347]]
Count 20000 Total [[45162.51237943]]
Count 30000 Total [[64427.98064641]]
Count 40000 Total [[86049.7890805]]
Count 50000 Total [[103616.91421075]]
Count 60000 Total [[127318.89566651]]
Count 70000 Total [[154207.5667278]]
Count 80000 Total [[180856.0298366]]
Count 90000 Total [[207825.60882472]]
Count 100000 Total [[225292.04581417]]
Count 110000 Total [[240232.58494873]]
Count 120000 Total [[255775.30529466]]
Count 130000 Total [[270887.50738186]]
Count 140000 Total [[286211.2338547]]
Count 150000 Total [[301560.43654101]]
Count 160000 Total [[316757.86769721]]
Count 170000 Total [[331835.8738666]]
Count 180000 Total [[347038.52015384]]
Count 190000 Total [[362358.20022462]]
Count 200000 Total [[377438.83295516]]
Count 210000 Total [[392926.35897205]]
Count 220000 Total [[412065.45632751]]
Count 230000 Total [[428039.16747175]]
Count 240000 Total [[445232.63500109]]
Count 250000 Total [[463363.56350758]]
Count 260000 Total [[478830.78023237]]
Coun

In [12]:
import multiprocessing as mp
import time
import numpy.linalg as LA

latent_dim = 5

In [70]:
# not enough RAM for both! for the ALS. dictionary maybe too much ram

# write multiprocessing with less RAM then? 
test_data = np.random.rand(1000, 100000)



In [75]:
def sum_of_row(r):
    return sum(r)

# cumulative sum 
pool = mp.Pool(mp.cpu_count())



t0 = time.time()
results = pool.map(sum_of_row, [row for row in test_data])
pool.close()
t1 = time.time()
print(t1-t0)
print(results[:10])

1.7160356044769287
[50070.83783296302, 50073.73481025633, 50086.41017229082, 49826.22335064556, 50070.81007714011, 49997.968458136456, 49956.87378708752, 50075.83199156748, 49914.99620557674, 49996.61793017107]


In [73]:
t0 = time.time()
results = [sum_row(row) for row in test_data]
t1 = time.time()
print(t1-t0)
print(results[:10])

5.86921763420105
[50070.83783296302, 50073.73481025633, 50086.41017229082, 49826.22335064556, 50070.81007714011, 49997.968458136456, 49956.87378708752, 50075.83199156748, 49914.99620557674, 49996.61793017107]


In [55]:
results = results.get()

In [ ]:
# https://stackoverflow.com/questions/20727375/multiprocessing-pool-slower-than-just-using-ordinary-functions
# speed up based on how you split up the data for sure! 


In [8]:
"""
Notes
randomly initialize q and p, or initialize as zeros initially
"""
def parallel_min_function_by_movie(movie, r, q, p, lamb): # can try by user later but probably worse. prove it.
    total = 0
    print(movie)
    for user in list(r[movie].keys())[:10]:
        rating = float(r[movie][user][0])
        if movie not in q:
            q[movie] = np.random.rand(latent_dim,1)
        if user not in p:
            p[user] = np.random.rand(latent_dim,1)
        prediction = np.dot(q[movie].T, p[user]) 
        norm_q_i = LA.norm(q[movie]) 
        norm_p_u = LA.norm(p[user])
        total += (rating-prediction)**2 + lamb*(norm_q_i + norm_p_u)
    return total
            

In [14]:


            
q = {}
p = {}



pool = mp.Pool(mp.cpu_count())
t0 = time.time()

results = pool.starmap_async(parallel_min_function_by_movie, [(movie, true_data, q, p, 0.1) for movie in list(true_data.keys())[:10]]).get()

pool.close()
t1 = time.time()
print(t1-t0)





Process ForkPoolWorker-36:
Process ForkPoolWorker-47:
Process ForkPoolWorker-33:
Process ForkPoolWorker-37:
Process ForkPoolWorker-44:


KeyboardInterrupt: 

Process ForkPoolWorker-45:
Process ForkPoolWorker-48:
Process ForkPoolWorker-43:
Process ForkPoolWorker-35:
Process ForkPoolWorker-42:
Process ForkPoolWorker-39:
Process ForkPoolWorker-46:
Process ForkPoolWorker-34:
Process ForkPoolWorker-41:
Process ForkPoolWorker-38:
Process ForkPoolWorker-40:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/kevin/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/kevin/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/home/kevin/anaconda3/lib/python3.7/multiprocessing

  File "/home/kevin/anaconda3/lib/python3.7/multiprocessing/queues.py", line 351, in get
    with self._rlock:
  File "/home/kevin/anaconda3/lib/python3.7/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
  File "/home/kevin/anaconda3/lib/python3.7/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
  File "/home/kevin/anaconda3/lib/python3.7/multiprocessing/queues.py", line 351, in get
    with self._rlock:
  File "/home/kevin/anaconda3/lib/python3.7/multiprocessing/queues.py", line 351, in get
    with self._rlock:
  File "/home/kevin/anaconda3/lib/python3.7/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
  File "/home/kevin/anaconda3/lib/python3.7/multiprocessing/queues.py", line 351, in get
    with self._rlock:
  File "/home/kevin/anaconda3/lib/python3.7/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
  File "/hom

In [13]:
q = {}
p = {}



pool = mp.Pool(mp.cpu_count())
t0 = time.time()

results = [parallel_min_function_by_movie(movie, true_data, q, p, 0.1) for movie in list(true_data.keys())[:10]]

pool.close()
t1 = time.time()
print(t1-t0)





1
2
3
4
5
6
7
8
9
10
0.032386064529418945


In [93]:
[list(movie_data.keys())[:10] for movie_data in [true_data[movie] for movie in list(true_data.keys())[:10]]]

[['1488844',
  '822109',
  '885013',
  '30878',
  '823519',
  '893988',
  '124105',
  '1248029',
  '1842128',
  '2238063'],
 ['2059652',
  '1666394',
  '1759415',
  '1959936',
  '998862',
  '2625420',
  '573975',
  '392722',
  '1401650',
  '988104'],
 ['1025579',
  '712664',
  '1331154',
  '2632461',
  '44937',
  '656399',
  '439011',
  '1436762',
  '1644750',
  '2031561'],
 ['1065039',
  '1544320',
  '410199',
  '732556',
  '1201419',
  '668811',
  '303948',
  '2418760',
  '848193',
  '950078'],
 ['1745265',
  '885013',
  '1997470',
  '30878',
  '840543',
  '2477414',
  '1409354',
  '589967',
  '264160',
  '2084362'],
 ['2031561',
  '1176140',
  '2336133',
  '1521836',
  '117277',
  '326587',
  '1961542',
  '1041552',
  '1678346',
  '643182'],
 ['951709',
  '585247',
  '2625420',
  '2322468',
  '2056324',
  '1969230',
  '90021',
  '2082629',
  '1876156',
  '1359575'],
 ['824097',
  '2630686',
  '644003',
  '785314',
  '243963',
  '229220',
  '1447783',
  '2580478',
  '2599087',
  '235

In [29]:
# create a user dictionary
user_dictionary = {}

for movie in true_data.keys():
        for user in true_data[movie].keys():
            if user in user_dictionary:
                user_dictionary[user][movie] = 1               
            else:
                user_dictionary[user] = {movie: 1} 


dict_keys